In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# Download the YOLOv3 files
!wget https://pjreddie.com/media/files/yolov3-spp.weights
!wget https://github.com/pjreddie/darknet/blob/master/cfg/yolov3-spp.cfg

--2023-02-10 10:16:08--  https://pjreddie.com/media/files/yolov3-spp.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252209544 (241M) [application/octet-stream]
Saving to: ‘yolov3-spp.weights’

yolov3-spp.weights  100%[===================>] 240.53M  74.3MB/s    in 3.5s    

2023-02-10 10:16:11 (69.7 MB/s) - ‘yolov3-spp.weights’ saved [252209544/252209544]

--2023-02-10 10:16:12--  https://github.com/pjreddie/darknet/blob/master/cfg/yolov3-spp.cfg
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘yolov3-spp.cfg’

yolov3-spp.cfg          [  <=>               ] 348.66K  1.12MB/s    in 0.3s    

2023-02-10 10:16:12 (1.12 MB/s) - ‘yolov3-spp.cfg’ saved [357024]



In [2]:
# Import libraries
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import imutils
import tensorflow as tf
from tensorflow import keras

In [5]:
# Load the image dataset
images_path = '/content/gdrive/MyDrive/image'
images = []
for filename in os.listdir(images_path):
    img = plt.imread(os.path.join(images_path, filename))
    img = cv2.resize(img, (224, 224)) # Resize to (224, 224)
    images.append(img)
images = np.array(images, dtype=np.float32) / 255.0 # Normalize the pixel values to [0, 1]

# Load the label dataset
labels_path = '/content/gdrive/MyDrive/label'
labels = []
for filename in os.listdir(labels_path):
    with open(os.path.join(labels_path, filename)) as f:
        line = f.readline().split()
        label = [float(x) for x in line]
        labels.append(label)
labels = np.array(labels, dtype=np.float32)

Split the data into training, validation, and testing sets

In [6]:
# training, validation, and testing sets - 8:1:1 
split_1 = len(images) * 0.8
split_2 = len(images) * 0.9

train_images, train_labels = images[:int(split_1)], labels[:int(split_1)]
val_images, val_labels = images[int(split_1):int(split_2)], labels[int(split_1):int(split_2)]
test_images, test_labels = images[int(split_2):], labels[int(split_2):]

In [4]:
model_YOLO = cv2.dnn.readNetFromDarknet('/content/yolov3-spp.cfg', '/content/yolov3-spp.weights')

error: ignored

In [7]:
# Define the YOLO model
model_YOLO = keras.models.Sequential()
model_YOLO.add(keras.layers.Conv2D(16, (3, 3), activation='linear', padding='same', input_shape=(224, 224, 3)))
model_YOLO.add(keras.layers.BatchNormalization())
model_YOLO.add(keras.layers.LeakyReLU(alpha=0.1))
model_YOLO.add(keras.layers.MaxPooling2D((2, 2), padding='same'))

model_YOLO.add(keras.layers.Conv2D(32, (3, 3), activation='linear', padding='same'))
model_YOLO.add(keras.layers.BatchNormalization())
model_YOLO.add(keras.layers.LeakyReLU(alpha=0.1))
model_YOLO.add(keras.layers.MaxPooling2D((2, 2), padding='same'))

model_YOLO.add(keras.layers.Conv2D(64, (3, 3), activation='linear', padding='same'))
model_YOLO.add(keras.layers.BatchNormalization())
model_YOLO.add(keras.layers.LeakyReLU(alpha=0.1))
model_YOLO.add(keras.layers.MaxPooling2D((2, 2), padding='same'))

model_YOLO.add(keras.layers.Conv2D(128, (3, 3), activation='linear', padding='same'))
model_YOLO.add(keras.layers.BatchNormalization())
model_YOLO.add(keras.layers.LeakyReLU(alpha=0.1))
model_YOLO.add(keras.layers.MaxPooling2D((2, 2), padding='same'))

model_YOLO.add(keras.layers.Flatten())
model_YOLO.add(keras.layers.Dense(128, activation='linear'))
model_YOLO.add(keras.layers.LeakyReLU(alpha=0.1))
model_YOLO.add(keras.layers.Dense(64, activation='linear'))
model_YOLO.add(keras.layers.LeakyReLU(alpha=0.1))
model_YOLO.add(keras.layers.Dense(5, activation='linear'))

model_YOLO.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 16)      448       
                                                                 
 batch_normalization (BatchN  (None, 224, 224, 16)     64        
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 224, 224, 16)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 32)      4640      
                                                                 
 batch_normalization_1 (Batc  (None, 112, 112, 32)     1

In [8]:
# Compile the YOLO model
model_YOLO.compile(optimizer='sgd', loss='mean_squared_error', metrics=['accuracy'])

In [9]:
# Train the YOLO model
history_YOLO = model_YOLO.fit(train_images, train_labels, epochs=10, validation_data=(val_images, val_labels))

Epoch 1/10
8/8 [==============================] - 30s 4s/step - loss: 7043.5381 - accuracy: 0.6773 - val_loss: 18514.4746 - val_accuracy: 0.7419
Epoch 2/10
8/8 [==============================] - 35s 4s/step - loss: 3155.9792 - accuracy: 0.7410 - val_loss: 17186.2305 - val_accuracy: 0.7419
Epoch 3/10
8/8 [==============================] - 26s 3s/step - loss: 2919.2400 - accuracy: 0.7410 - val_loss: 13202.1250 - val_accuracy: 0.7419
Epoch 4/10
8/8 [==============================] - 28s 4s/step - loss: 2686.8853 - accuracy: 0.7410 - val_loss: 12131.5508 - val_accuracy: 0.7419
Epoch 5/10
8/8 [==============================] - 29s 4s/step - loss: 2588.6050 - accuracy: 0.7410 - val_loss: 9705.0488 - val_accuracy: 0.7419
Epoch 6/10
8/8 [==============================] - 29s 4s/step - loss: 2576.5605 - accuracy: 0.7410 - val_loss: 9041.5117 - val_accuracy: 0.7419
Epoch 7/10
8/8 [==============================] - 25s 3s/step - loss: 2453.6626 - accuracy: 0.7410 - val_loss: 8543.6992 - val_accur

In [10]:
# Evaluate the YOLO model on the test data
test_loss_YOLO, test_accuracy_YOLO = model_YOLO.evaluate(test_images, test_labels, verbose=0)
print("Test loss:", test_loss_YOLO)
print("Test accuracy:", test_accuracy_YOLO)

Test loss: 5856.015625
Test accuracy: 0.59375
